# Imports and Paths

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import shutil
from skimage import io, transform

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt

In [2]:
PATH = '/data/msnow/nih_cxr/'

# Load the Data

In [3]:
df = pd.read_csv(f'{PATH}Data_Entry_2017.csv')
df.shape

(112120, 11)

In [4]:
df.head()

,image,labels,followup,id,age,gender,view,image_width,image_height,pixelspacing_x,pixelspacing_y
0,00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143
1,00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143
2,00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168
3,00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171
4,00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143


In [5]:
img_list = os.listdir(f'{PATH}images')
len(img_list)

112120

## Collate the data

In [6]:
df_pa = df.loc[df.view=='PA',:]
df_pa.reset_index(drop=True, inplace=True)

In [7]:
trn_sz = int(df_pa.shape[0]/2)
df_pa_trn = df_pa.loc[:trn_sz,:]
df_pa_tst = df_pa.loc[trn_sz:,:]

In [50]:
df_pa_tst.shape

(33655, 11)

In [8]:
pneumo = []
for i,v in df_pa_trn.iterrows():
    if "pneumo" in v['labels'].lower():
        pneumo.append('pneumo')
    else:
        pneumo.append('no pneumo')
df_pa_trn['pneumo'] = pneumo

/data/msnow/miniconda3/envs/data_sci/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [9]:
pneumo = []
for i,v in df_pa_tst.iterrows():
    if "pneumo" in v['labels'].lower():
        pneumo.append(pneumo)
    else:
        pneumo.append('no pneumo')
df_pa_tst['pneumo'] = pneumo

/data/msnow/miniconda3/envs/data_sci/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [10]:
df_pa_trn.shape

(33656, 12)

Copy images to train and test folders

In [55]:
# dst = os.path.join(PATH,'trn')
# src = os.path.join(PATH,'images')
# for i,v in df_pa_trn.iterrows():
#     src2 = os.path.join(src,v.image)    
#     shutil.copy2(src2,dst)

In [56]:
# dst = os.path.join(PATH,'tst')
# src = os.path.join(PATH,'images')
# for i,v in df_pa_tst.iterrows():
#     src2 = os.path.join(src,v.image)    
#     shutil.copy2(src2,dst)

# Create the Dataset and Dataloader

In [11]:
class TDataset(Dataset):

    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            df       (dataframe): df with all the annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
#         self.landmarks_frame = pd.read_csv(csv_file)
        self.df = df
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,self.df.image[idx])
        image = io.imread(img_name)
        categ = self.df.pneumo[idx]

        return image, categ

In [13]:
aa = trainset[0]

In [12]:
trainset = TDataset(df_pa_trn,f'{PATH}trn')
testset = TDataset(df_pa_tst,f'{PATH}tst')

In [21]:
trainloader = DataLoader(trainset, batch_size=4,shuffle=True, num_workers=4)
testloader = DataLoader(testset, batch_size=4,shuffle=False, num_workers=4)

In [17]:
aa[0].shape

(1024, 1024)

 # Define and train a CNN

In [60]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        # 6 input image channel, 16 output channels, 5x5 square convolution kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        print(f'input shape {x.shape}')
        x = self.pool(F.relu(self.conv1(x)))
        print(f'Lin (1,6,5) + relu + pool shape {x.shape}')
        x = self.pool(F.relu(self.conv2(x)))
        print(f'Lin (6,16,5) + relu + pool shape {x.shape}')
        x = x.view(x.shape[0],-1)
        print(f'reshape shape {x.shape}')
#         x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
        return x


net = Net()

In [35]:
input = torch.randn(1, 1, 1024,1024)
out = net(input)
# print(out)

torch.Size([1, 1, 1024, 1024])


In [103]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [22]:
for i, data in enumerate(trainloader, 0):
    break

In [23]:
inputs, labels = data

In [61]:
tst = inputs.view(-1,1,1024,1024)
tst = tst.type('torch.FloatTensor')
out = net(tst)

input shape torch.Size([4, 1, 1024, 1024])
Lin (1,6,5) + relu + pool shape torch.Size([4, 6, 510, 510])
Lin (6,16,5) + relu + pool shape torch.Size([4, 16, 253, 253])
reshape shape torch.Size([4, 1024144])


RuntimeError: size mismatch, m1: [4 x 1024144], m2: [400 x 120] at /opt/conda/conda-bld/pytorch-cpu_1524582300956/work/aten/src/TH/generic/THTensorMath.c:2033

In [57]:
16*253*253

1024144

In [46]:
tst.shape

torch.Size([4, 1, 1024, 1024])

torch.Size([4, 1, 1024, 1024])


In [129]:
# tst = inputs[:,None,:,:]
tst.type(torch.FloatTensor)

tensor([[[[  94.,   80.,   67.,  ...,    1.,    1.,    0.],
          [  78.,   62.,   50.,  ...,    1.,    1.,    0.],
          [  63.,   50.,   40.,  ...,    1.,    1.,    0.],
          ...,
          [  33.,   28.,   24.,  ...,    2.,    1.,    0.],
          [  36.,   30.,   28.,  ...,    2.,    1.,    0.],
          [  39.,   34.,   30.,  ...,    3.,    2.,    0.]]],


        [[[ 141.,  139.,  140.,  ...,    2.,    1.,    0.],
          [ 137.,  133.,  134.,  ...,    1.,    1.,    0.],
          [ 134.,  135.,  133.,  ...,    0.,    0.,    0.],
          ...,
          [  24.,   15.,   10.,  ...,   51.,   34.,   10.],
          [  25.,   16.,   11.,  ...,   62.,   41.,   17.],
          [  26.,   16.,   11.,  ...,   71.,   49.,   20.]]],


        [[[ 114.,  113.,  109.,  ...,    1.,    1.,    0.],
          [ 113.,  110.,  108.,  ...,    1.,    1.,    0.],
          [ 112.,  108.,  106.,  ...,    1.,    1.,    0.],
          ...,
          [ 228.,  231.,  227.,  ...,    4.,   

In [130]:
type(tst)

torch.Tensor

In [16]:
list(net.parameters())[0].size()

NameError: name 'net' is not defined

In [131]:
net(tst)

torch.Size([4, 1, 1024, 1024])


RuntimeError: thnn_conv2d_forward is not implemented for type torch.ByteTensor

In [123]:
conv1_tst(tst)

RuntimeError: thnn_conv2d_forward is not implemented for type torch.ByteTensor

In [104]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

RuntimeError: expected stride to be a single integer value or a list of 1 values to match the convolution dimensions, but got stride=[1, 1]